In [45]:
# Imports
import polars as pl
import altair as alt

In [37]:
def rdd_bdgt_df():
    return pl.read_csv("data/OECD.IEA,RDDPUBLIC,1.0,filtered,2025-10-16 03-48-20.csv").drop(
        ["STRUCTURE", "STRUCTURE_ID", "STRUCTURE_NAME", "ACTION", "Decimals", 
        "FREQUENCY", "Frequency", "RDD_TYPE", "Type", "Time Period", 
        "RDD_SECTOR", "Sector", "Confidential Status"])

In [38]:
def rdd_bdgt_oecd_europe_df():
    oecd_europe_countries = [
    "AUSTRIA", "BELGIUM", "CZECH", "DENMARK", "ESTONIA", 
    "FINLAND", "FRANCE", "GERMANY", "GREECE", "HUNGARY",
    "IRELAND", "ITALY", "LITHUANIA", 
    "LUXEMBOURG", "NETHERLANDS", "NORWAY", "POLAND", "PORTUGAL", 
    "SLOVAKIA", "SPAIN", "SWEDEN", "SWITZERLAND", 
    "UNITED KINGDOM"
    ]

    return rdd_bdgt_df().filter(pl.col("COUNTRY").is_in(oecd_europe_countries))

In [63]:
rdd_bdgt_oecd_europe_df()

COUNTRY,Country/Region,RDD_TECH,Technology,UNIT,Unit,TIME_PERIOD,OBS_VALUE,Observation value,QUALIFIER,Qualifier,CONF_STATUS,UNIT_MULT,Unit multiplier,DECIMALS
str,str,str,str,str,str,i64,f64,str,str,str,str,i64,str,i64
"""IRELAND""","""Ireland""","""FOSSILFUEL""","""Group 2: Fossil fuels""","""NC_N""","""National currency (nominal)""",2020,0.911,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
"""IRELAND""","""Ireland""","""FOSSILFUEL""","""Group 2: Fossil fuels""","""NC_N""","""National currency (nominal)""",2021,0.821,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
"""IRELAND""","""Ireland""","""FOSSILFUEL""","""Group 2: Fossil fuels""","""NC_N""","""National currency (nominal)""",2022,0.712,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
"""IRELAND""","""Ireland""","""FOSSILFUEL""","""Group 2: Fossil fuels""","""NC_N""","""National currency (nominal)""",2023,0.458,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
"""HUNGARY""","""Hungary""","""EFFICIENCY""","""Group 1: Energy efficiency""","""NC_N""","""National currency (nominal)""",2020,2365.471,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""DENMARK""","""Denmark""","""UNALLOC""","""Group 8: Unallocated""","""NC_N""","""National currency (nominal)""",2024,0.0,null,"""P""","""Provisional Value""","""F""",6,"""Millions""",3
"""FRANCE""","""France""","""RENEWABLE""","""Group 3: Renewable energy sour…","""NC_N""","""National currency (nominal)""",2020,194.931,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3
"""FRANCE""","""France""","""RENEWABLE""","""Group 3: Renewable energy sour…","""NC_N""","""National currency (nominal)""",2021,206.842,null,"""I""","""Imputed Value""","""F""",6,"""Millions""",3


In [84]:
rdd_bdgt_oecd_europe_df().filter(
    (pl.col("COUNTRY") == "HUNGARY") & (pl.col("TIME_PERIOD") == 2021)
).group_by(
    "RDD_TECH", "TIME_PERIOD"
).agg(
    pl.col("OBS_VALUE").sum()
).sort("TIME_PERIOD", "RDD_TECH")

RDD_TECH,TIME_PERIOD,OBS_VALUE
str,i64,f64
"""EFFICIENCY""",2021,7586.896
"""FOSSILFUEL""",2021,230.354
"""HGENCELL""",2021,1829.159
"""NUCLEAR""",2021,8.493
"""OTHERPANDS""",2021,9271.286
"""OTHERTECH""",2021,25.0
"""RENEWABLE""",2021,4153.828
"""TOTAL""",2021,23105.017
"""UNALLOC""",2021,0.0


In [86]:
rdd_bdgt_oecd_europe_df().filter(
    (pl.col("COUNTRY") == "HUNGARY") & (pl.col("TIME_PERIOD") == 2022)
).group_by(
    "RDD_TECH", "TIME_PERIOD"
).agg(
    pl.col("OBS_VALUE").sum()
).sort("TIME_PERIOD", "RDD_TECH")

RDD_TECH,TIME_PERIOD,OBS_VALUE
str,i64,f64
"""EFFICIENCY""",2022,790.765
"""FOSSILFUEL""",2022,66.979
"""HGENCELL""",2022,0.0
"""NUCLEAR""",2022,1141.365
"""OTHERPANDS""",2022,461.941
"""OTHERTECH""",2022,12.87
"""RENEWABLE""",2022,193.385
"""TOTAL""",2022,2667.305
"""UNALLOC""",2022,0.0


In [76]:

alt.Chart(rdd_bdgt_oecd_europe_df().filter(pl.col("Technology") != "Total")).mark_bar().encode(
    x="COUNTRY:N",
    y="OBS_VALUE:Q",
    xOffset="TIME_PERIOD:N",
    color="TIME_PERIOD:N"
)


alt.Chart(...)

In [225]:
def china_solar_exports_df():
    df = pl.read_csv("data/mart_solar_exports_full_release_monthly.csv")
    
    # This is the correct pattern
    return (
        df.with_columns(
            pl.col("Date").str.to_date()  # 1. Update the Date column
        )
        .filter(
            # 2. THEN, filter the entire DataFrame
            pl.col("Area").is_in([
                "Other Oceania", 
                "Other Africa", 
                "Other Latin America"
            ]).not_() # (or use the ~ operator)
        )
    )

In [226]:
def china_solar_exports_countries_df():
    return china_solar_exports_df().filter(
        pl.col("Area type") == "Country or economy"
    )

china_solar_exports_countries_df()

Area,Date,Area type,Region,Commodity code,Commodity type,Amount (items),Amount (kg),Amount (USD),Commodity price,Price unit,Capacity (MW),Cumulative capacity (MW)
str,date,str,str,i64,str,i64,f64,f64,f64,str,f64,f64
"""Afghanistan""",2017-01-01,"""Country or economy""","""Asia""",85414020,"""Panels""",49433,333324.0,1.663024e6,0.38,"""USD/W""",4.38,4.38
"""Afghanistan""",2017-02-01,"""Country or economy""","""Asia""",85414020,"""Panels""",5452,82456.0,337723.0,0.38,"""USD/W""",0.89,5.27
"""Afghanistan""",2017-03-01,"""Country or economy""","""Asia""",85414020,"""Panels""",51403,940507.0,3.477201e6,0.38,"""USD/W""",9.15,14.42
"""Afghanistan""",2017-04-01,"""Country or economy""","""Asia""",85414020,"""Panels""",100875,1.306053e6,4.783758e6,0.38,"""USD/W""",12.59,27.0
"""Afghanistan""",2017-05-01,"""Country or economy""","""Asia""",85414020,"""Panels""",67325,786843.0,2.823671e6,0.38,"""USD/W""",7.43,34.44
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zimbabwe""",2025-06-01,"""Country or economy""","""Africa""",85414300,"""Panels""",15891,351968.0,586375.0,0.09,"""USD/W""",6.18,470.44
"""Zimbabwe""",2025-07-01,"""Country or economy""","""Africa""",85414200,"""Cells""",1611,26079.0,48306.0,0.04,"""USD/W""",1.32,21.78
"""Zimbabwe""",2025-07-01,"""Country or economy""","""Africa""",85414300,"""Panels""",36213,733391.0,1.441831e6,0.09,"""USD/W""",15.22,485.66


In [227]:
def china_solar_exports_regions_df(region: str):
    df = china_solar_exports_df().filter(pl.col("Area type") == "Region")

    if region == "world":
        return df.filter(pl.col("Area") == "World")
    elif region == "regions":
          return df.filter(pl.col("Area") != "World")
    else:
        return df.filter(pl.col("Area") == region)
    
def china_solar_exports_monthly(region: str):
    df = china_solar_exports_regions_df(region)

    result = (
        df.group_by(["Area", "Date", "Commodity type"]) 
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .rename({"Area": "Region"})
        .sort(["Region", "Date", "Commodity type"])
    )

    return result

def china_solar_exports_monthly_by_region(region: str):
    
    # 1. Get the detailed data (with commodity types)
    detailed_df = china_solar_exports_monthly(region)
    
    # 2. Add a final aggregation to sum up the commodity types
    region_agg_df = (
        detailed_df.group_by(["Region", "Date"])
        .agg(pl.col("Total Capacity (MW)").sum()) # Sums the commodities
        .sort(["Region", "Date"])
    )
    return region_agg_df


In [228]:
china_solar_exports_monthly_by_region("regions") 

Region,Date,Total Capacity (MW)
str,date,f64
"""ASEAN""",2017-01-01,32.31
"""ASEAN""",2017-02-01,29.09
"""ASEAN""",2017-03-01,49.94
"""ASEAN""",2017-04-01,48.92
"""ASEAN""",2017-05-01,44.74
…,…,…
"""Oceania""",2025-04-01,601.02
"""Oceania""",2025-05-01,731.41
"""Oceania""",2025-06-01,698.98


In [170]:
chart = (
    alt.Chart(china_solar_exports_monthly("world"))
    .mark_bar(color="#1f77b4")
    .encode(
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(
                format="%Y",        # show only years
                tickCount="year",   # one tick per year
            )
        ),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        tooltip=[
            alt.Tooltip("Date:T", title="Month"),
            alt.Tooltip("Total Capacity (MW):Q", title="Total Capacity (MW)")
        ]
    )
    .properties(width=700, height=400, title="China Solar Exports - Monthly Capacity (World)")
)

chart

alt.Chart(...)

In [188]:
import altair as alt

# 1. Load the data (same as your example)
df_world_details = china_solar_exports_monthly("world")

# 2. Create the stacked BAR chart
stacked_bar_chart = (
    alt.Chart(df_world_details)
    .mark_bar()  # <-- The only change is from mark_area()
    .encode(
        # X-axis: Time (Date)
        x=alt.X(
            "Date:T",
            title="Year",
            # We remove the tickCount to let Altair show monthly bars
            axis=alt.Axis(format="%Y") 
        ),
        
        # Y-axis: Total capacity, stacked to zero
        y=alt.Y(
            "Total Capacity (MW):Q", 
            title="Total Capacity (MW)", 
            stack="zero"  # Stacks the bars
        ),
        
        # Color: The different technology types
        color=alt.Color(
            "Commodity type:N", 
            title="Technology"
        ),
        
        # Tooltip: Show details on hover
        tooltip=[
            alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
        ]
    )
    .properties(
        title="China's Monthly Solar Exports by Technology (World)",
        width=750,
        height=400
    )
    .interactive()  # Allows zooming and panning
)

stacked_bar_chart

alt.Chart(...)

In [183]:
import altair as alt

# 1. Load the data using the function that includes commodity type
# We use "world" to get the total global export picture
df_world_details = china_solar_exports_monthly("world")

# 2. Create the stacked area chart
stacked_area_chart = (
    alt.Chart(df_world_details)
    .mark_area()
    .encode(
        # X-axis: Time (Date)
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        
        # Y-axis: Total capacity, stacked to zero
        y=alt.Y(
            "Total Capacity (MW):Q", 
            title="Total Capacity (MW)", 
            stack="zero"  # This tells Altair to stack the areas
        ),
        
        # Color: The different technology types
        color=alt.Color(
            "Commodity type:N", 
            title="Technology"
        ),
        
        # Tooltip: Show details on hover
        tooltip=[
            alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
        ]
    )
    .properties(
        title="China's Monthly Solar Exports by Technology (World)",
        width=750,
        height=400
    )
    .interactive()  # Allows zooming and panning
)

stacked_area_chart

alt.Chart(...)

In [179]:
base = alt.Chart(china_solar_exports_monthly_by_region("regions")).encode(
    x=alt.X(
        "Date:T",
        title="Year",
        axis=alt.Axis(format="%Y", tickCount="year")
    ),
    y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
    color=alt.Color("Region:N", title="Region")
)

# Line chart
lines = base.mark_line().encode(
    tooltip=["Region", "Date:T", "Total Capacity (MW):Q"]
)

# Points for the last value of each region
last_points = (
    base.transform_aggregate(
        last_date="max(Date)",
        groupby=["Region"]
    )
    .transform_filter("datum.Date == datum.last_date")
    .mark_point(filled=True, size=60)
)

# Labels at the last point
labels = (
    base.transform_aggregate(
        last_date="max(Date)",
        groupby=["Region"]
    )
    .transform_filter("datum.Date == datum.last_date")
    .mark_text(align="left", dx=5)
    .encode(text="Region:N")
)

# Combine layers
chart = (
    (lines + last_points + labels)
    .properties(
        width=750,
        height=400,
        title="China Solar Exports - Monthly Capacity by Region"
    )
    .configure_axis(
        grid=False
    )
)

chart

alt.LayerChart(...)

In [195]:
# 1. Load the regional data
regions_df = china_solar_exports_monthly_by_region("regions")

# 2. Load the world data
world_df = china_solar_exports_monthly_by_region("world")

# 3. Combine them into a single DataFrame
combined_df = pl.concat([regions_df, world_df])

#######

# Create the base chart (same as before)
base = alt.Chart(combined_df).encode(
    x=alt.X(
        "Date:T",
        title="Year",
        axis=alt.Axis(format="%Y", tickCount="year")
    ),
    y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
    color=alt.Color("Region:N", title="Region")
)

# --- MODIFIED Line chart ---
lines = base.mark_line().encode(
    tooltip=["Region", "Date:T", "Total Capacity (MW):Q"],
    strokeWidth=alt.condition(
        alt.datum.Region == "World",
        alt.value(3.5),
        alt.value(1.5)
    ),
    strokeDash=alt.condition(
        alt.datum.Region == "World",
        alt.value([5, 3]),
        alt.value([1, 0])
    )
)

# --- MODIFIED Points ---
last_points = (
    base.transform_aggregate(
        last_date="max(Date)",
        groupby=["Region"]
    )
    .transform_filter("datum.Date == datum.last_date")
    .mark_point(filled=True)
    .encode(
        size=alt.condition(
            alt.datum.Region == "World",
            alt.value(100),
            alt.value(60)
        )
    )
)

# --- REPLACEMENT LABELS (THE FIX) ---

# 1. Create the labels for all regions *except* "World"
labels_normal = (
    base.transform_aggregate(
        last_date="max(Date)",
        groupby=["Region"]
    )
    .transform_filter("datum.Date == datum.last_date")
    .transform_filter(alt.datum.Region != "World")  # <-- Filter out "World"
    .mark_text(align="left", dx=5, fontWeight="normal") # <-- Set weight directly
    .encode(
        text="Region:N",
        color=alt.value("black")  # Set all labels to black
    )
)

# 2. Create the label *only* for "World"
labels_bold = (
    base.transform_aggregate(
        last_date="max(Date)",
        groupby=["Region"]
    )
    .transform_filter("datum.Date == datum.last_date")
    .transform_filter(alt.datum.Region == "World")  # <-- Filter *for* "World"
    .mark_text(align="left", dx=5, fontWeight="bold")  # <-- Set weight to bold
    .encode(
        text="Region:N",
        color=alt.value("black") # Set label to black
    )
)

# --- Combine all layers ---
styled_chart = (
    # Add both label layers
    (lines + last_points + labels_normal + labels_bold)
    .properties(
        width=750,
        height=400,
        title="China Solar Exports - Monthly Capacity by Region (with World Total)"
    )
    .configure_axis(
        grid=False
    )
)

styled_chart

alt.LayerChart(...)

In [187]:
import altair as alt
import polars as pl

# 1. Load the data
# We still use "regions" to get the breakdown
df_regions_details = china_solar_exports_monthly("regions")

# 2. Create the Grouped Bar Chart
grouped_bar_chart = (
    alt.Chart(df_regions_details)
    .mark_bar()
    .encode(
        # On the x-axis, we'll group the technologies.
        # We hide the axis labels for a cleaner look.
        x=alt.X("Commodity type:N", title=None, axis=None),
        
        # The y-axis is the total sum for that technology
        y=alt.Y(
            "sum(Total Capacity (MW)):Q",
            title="Total Capacity (MW)"
        ),
        
        # Color the bars by the technology type
        color=alt.Color(
            "Commodity type:N",
            title="Technology"
        ),
        
        # This is the key: we create a "column" for each region.
        # This groups the three bars together.
        column=alt.Column(
            "Region:N",
            title="Region",
            header=alt.Header(titleOrient="bottom", labelOrient="bottom", titlePadding=5),
        ),
        
        # Add a clear tooltip
        tooltip=[
            "Region:N",
            "Commodity type:N",
            alt.Tooltip(
                "sum(Total Capacity (MW)):Q",
                title="Total Capacity (MW)",
                format=","
            ),
        ],
    )
    .properties(
        title="Total Solar Exports by Technology and Region",
        width=alt.Step(25),  # This sets the width of each *individual bar*
        height=300
    )
    .configure_facet(
        spacing=15  # Adds a little space between each region's group
    )
    .configure_view(
        stroke=None  # Removes the outer border
    )
)

grouped_bar_chart

alt.Chart(...)

In [189]:
import altair as alt
import polars as pl

# 1. Load the data
df_regions_details = china_solar_exports_monthly("regions")

# 2. Create the faceted 100% stacked bar chart
wrapped_normalized_bar_chart = (
    alt.Chart(df_regions_details)
    .mark_bar()
    .encode(
        # CHANGE 1: We no longer need the X-axis encoding for the region.
        # The facet itself will handle separating the regions.
        
        # Y-axis: Sum of capacity, normalized to 100%
        y=alt.Y(
            "sum(Total Capacity (MW)):Q",
            title="Share of Exports",
            stack="normalize",  # This creates the 100% stack
            axis=alt.Axis(format="%") # Format the axis as percentage
        ),
        
        # Color: Technology type
        color=alt.Color(
            "Commodity type:N",
            title="Technology"
        ),

        # Tooltip: Show details
        tooltip=[
            "Region:N",
            "Commodity type:N",
            alt.Tooltip(
                "sum(Total Capacity (MW)):Q",
                title="Total Capacity (MW)",
                format=","
            )
        ],

        # CHANGE 2: Replace 'column' with 'facet' to enable wrapping
        facet=alt.Facet(
            "Region:N",
            columns=4  # <-- Set how many charts you want per row
        )
    )
    .properties(
        title="Composition of China's Solar Exports by Region (All Time)",
        # CHANGE 3: Adjust width/height for each individual chart in the grid
        width=150,
        height=300
    )
)

wrapped_normalized_bar_chart

alt.Chart(...)

In [208]:
china_solar_exports_countries_df()

Area,Date,Area type,Region,Commodity code,Commodity type,Amount (items),Amount (kg),Amount (USD),Commodity price,Price unit,Capacity (MW),Cumulative capacity (MW)
str,date,str,str,i64,str,i64,f64,f64,f64,str,f64,f64
"""Afghanistan""",2017-01-01,"""Country or economy""","""Asia""",85414020,"""Panels""",49433,333324.0,1.663024e6,0.38,"""USD/W""",4.38,4.38
"""Afghanistan""",2017-02-01,"""Country or economy""","""Asia""",85414020,"""Panels""",5452,82456.0,337723.0,0.38,"""USD/W""",0.89,5.27
"""Afghanistan""",2017-03-01,"""Country or economy""","""Asia""",85414020,"""Panels""",51403,940507.0,3.477201e6,0.38,"""USD/W""",9.15,14.42
"""Afghanistan""",2017-04-01,"""Country or economy""","""Asia""",85414020,"""Panels""",100875,1.306053e6,4.783758e6,0.38,"""USD/W""",12.59,27.0
"""Afghanistan""",2017-05-01,"""Country or economy""","""Asia""",85414020,"""Panels""",67325,786843.0,2.823671e6,0.38,"""USD/W""",7.43,34.44
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zimbabwe""",2025-06-01,"""Country or economy""","""Africa""",85414300,"""Panels""",15891,351968.0,586375.0,0.09,"""USD/W""",6.18,470.44
"""Zimbabwe""",2025-07-01,"""Country or economy""","""Africa""",85414200,"""Cells""",1611,26079.0,48306.0,0.04,"""USD/W""",1.32,21.78
"""Zimbabwe""",2025-07-01,"""Country or economy""","""Africa""",85414300,"""Panels""",36213,733391.0,1.441831e6,0.09,"""USD/W""",15.22,485.66


In [206]:
import polars as pl

def china_solar_exports_by_country():
    """
    Aggregates total capacity by country, using only the 'Area' (name).
    """
    # Filter for rows where the Area type is "Country"
    df = china_solar_exports_countries_df()
    
    # Group by only the country name ("Area")
    country_totals = (
        df.group_by("Area") 
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
    )
    
    return country_totals

In [229]:
import altair as alt
from vega_datasets import data

# 1. Load the base map shapes
world_map_data = alt.topo_feature(data.world_110m.url, 'countries')

# 2. Load the country location helper (has lat/lon and 'name')
country_locations_url = data.countries.url

# 3. Load your aggregated export data (which now *only* has 'Area')
export_data = pl.read_csv("data/china_solar_exports_by_country.csv")

# --- Build the Chart Layers ---

# Layer 1: The grey background map
base_map = (
    alt.Chart(world_map_data)
    .mark_geoshape(
        fill="lightgray",
        stroke="white",
        strokeWidth=0.5
    )
    .project(type="mercator")
)

# Layer 2: The circles (JOINING ON NAME)
symbols = (
    alt.Chart(export_data)  # We chart your data
    .mark_circle()
    .transform_lookup(
        # Look up using the 'Area' (name) from your data
        lookup="Area", 
        
        # Join it with the countries helper file
        from_=alt.LookupData(
            country_locations_url, 
            key="name",  # <-- The matching key is 'name'
            fields=["latitude", "longitude"] # Get the lat/lon
        )
    )
    # Only show countries that are in your export data
    .transform_filter(alt.datum["Total Capacity (MW)"] > 0) 
    .encode(
        # Use the lat/lon we just looked up
        longitude="longitude:Q",
        latitude="latitude:Q",
        
        size=alt.Size(
            "Total Capacity (MW):Q",
            title="Capacity (MW)",
            scale=alt.Scale(type="log", range=[10, 1000]) 
        ),
        
        color=alt.value("steelblue"),
        
        tooltip=[
            alt.Tooltip("Area:N", title="Country"),
            alt.Tooltip("Total Capacity (MW):Q", title="Total Capacity (MW)", format=",")
        ]
    )
)

# Combine the layers
symbol_map = (
    (base_map + symbols)
    .properties(
        title="China's Total Solar Exports by Country",
        width=800,
        height=450
    )
    .interactive()
)

symbol_map

alt.LayerChart(...)